### Environment Setup

In [ ]:
%pip install -q -r requirements.txt

Note: you may need to restart the kernel to use updated packages.


In [1]:
from dotenv import load_dotenv
_ = load_dotenv()

In the Basics notebook, we created templates and invoked them and finally feed the result to the LLM. We can make this easier by using Langchain expression language (LCEL) which is very similar to the piping in the Linux shells.

In [2]:
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(model="gpt-4o-mini", temperature=0)

In [3]:
out = llm.invoke("Hello, who are you?")
print(out.content)

Hello! I’m an AI language model created by OpenAI. I'm here to assist you with information, answer questions, and engage in conversation. How can I help you today?


### Templates

Multiple invokes without chaining

In [4]:
from langchain_core.prompts import ChatPromptTemplate

template = ChatPromptTemplate.from_template("""
Rate a product score of based on the following user comment from 0 to 5 where 5 is the maximum and 0 is the minimum.
comment: {comment}
""")
llm.invoke(template.invoke({"comment": "I love this product, it is the best thing ever!"}))

AIMessage(content='Based on the user comment, I would rate the product a score of 5. The enthusiasm and positive language indicate a very high level of satisfaction.', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 31, 'prompt_tokens': 52, 'total_tokens': 83, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_0aa8d3e20b', 'finish_reason': 'stop', 'logprobs': None}, id='run-0c2248ce-569c-4572-a5c4-69c59a8a3f20-0', usage_metadata={'input_tokens': 52, 'output_tokens': 31, 'total_tokens': 83, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 0}})

Single invoke with chaining  

In the chaining, we use the `|` operator. When the input is provided to the chain, template being the first layer in chain, will be invoked to create the input to the next layer by filling the template. Next layer in the chain, LLM, will be invoked with the output of the previous layer and so on.

In [5]:
templated_llm = template | llm

In [6]:
templated_llm.invoke({"comment": "I love this product, it is the best thing ever!"})

AIMessage(content='Based on the user comment, I would rate the product a score of 5. The enthusiasm and positive language indicate a very high level of satisfaction.', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 31, 'prompt_tokens': 52, 'total_tokens': 83, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_f2cd28694a', 'finish_reason': 'stop', 'logprobs': None}, id='run-ef1efb70-2892-4347-ac92-10d6184b2441-0', usage_metadata={'input_tokens': 52, 'output_tokens': 31, 'total_tokens': 83, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 0}})

In [7]:
# again if there's only one variable, you can just pass it directly
templated_llm.invoke("I love this product, it is the best thing ever!")

AIMessage(content='Based on the user comment, I would rate the product a score of 5. The enthusiasm and positive language indicate a very high level of satisfaction.', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 31, 'prompt_tokens': 52, 'total_tokens': 83, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_d02d531b47', 'finish_reason': 'stop', 'logprobs': None}, id='run-68190604-3242-4e46-99a7-29af7f5605fd-0', usage_metadata={'input_tokens': 52, 'output_tokens': 31, 'total_tokens': 83, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 0}})

### Runnable

At the heart of langchain chains is the runnable abstraction. Many classes implement the runnable interface that allows them to be invoked in the chain. 
A custom runnable function can be created using RunnableLamdba.

In [8]:
# A runnable that returns the length of input string when invoked
from langchain_core.runnables import RunnableLambda

str_uppercaser = RunnableLambda(lambda s: s.upper())
str_uppercaser.invoke("Hello world")

'HELLO WORLD'

In [9]:
# A runnable that returns reversed input string when invoked
str_reverser = RunnableLambda(lambda s: s[::-1])

In [10]:
# A chained runnable that returns reversed and uppercased input string when invoked
(str_reverser | str_uppercaser).invoke("Hello world")

'DLROW OLLEH'

In [12]:
# A lambda can be directly passed to the chain as well, it will be converted to a RunnableLambda implicitly.
chain = str_reverser | \
        (lambda s: {"length": len(s), "input": s})
chain.invoke("Hello world")

{'length': 11, 'input': 'dlrow olleH'}

#### RunnablePassthrough

A RunnablePassthrough is a runnable that simply passes the input to the output.

In [13]:
from langchain_core.runnables import RunnableLambda, RunnablePassthrough

# this way it's not useful but to demonstrate.
(str_reverser | RunnablePassthrough()).invoke("Hello world")

'dlrow olleH'

In [14]:
# it can be used to extend input&output dictionaries.
runnable1 = RunnableLambda(lambda x: x["foo"] + 7)
chain = RunnablePassthrough.assign(bar=runnable1)
chain.invoke({"foo": 10})

{'foo': 10, 'bar': 17}

In [93]:
# A chained runnable that returns the filled template and llm response to this filled template.
# It also demonstrates how to create a Runnable(Parallel) from a dictionaru
templated_llm = template | {
    'question': RunnablePassthrough(), # template output is passed through
    'llm_response': llm # Parsing logic
}


In [15]:
templated_llm.invoke("Tell me a joke about a chicken")

AIMessage(content='Based on the user comment "Tell me a joke about a chicken," it seems the user is looking for humor or entertainment rather than providing feedback on a product. Therefore, I would rate the product score as 2. This indicates a low level of satisfaction or relevance to the product, as the comment does not provide any specific feedback or critique.', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 70, 'prompt_tokens': 48, 'total_tokens': 118, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_0aa8d3e20b', 'finish_reason': 'stop', 'logprobs': None}, id='run-c401a788-aa73-40f0-b021-04138e950212-0', usage_metadata={'input_tokens': 48, 'output_tokens': 70, 'total_tokens': 118, 'input_token_details': {'audio': 0, 

#### RunnableParallel

Runs the runnables in parallel and combines the results in the given parameters. Either a dictionary or named arguments can be passed. Function names or lambdas can be passed, they are converted to RunnableLambda implicitly.

In [16]:
# passing dictionary directly
from langchain_core.runnables import RunnableParallel

RunnableParallel({
    "len": lambda s: len(s),
    "reverse": lambda s: s[::-1]
}).invoke("Hello world")

{'len': 11, 'reverse': 'dlrow olleH'}

In [17]:
# passing named args
RunnableParallel(
    len=lambda s: len(s),
    reverse=lambda s: s[::-1]
).invoke("Hello world")

{'len': 11, 'reverse': 'dlrow olleH'}

Does it really execute in parallel?

In [18]:
def long_running_func(s):
    import time
    for i in range(10):
        time.sleep(1)
        print(f"Processing '{s}', step{i+1}")
    return f"Processed {s}"

In [19]:
def another_long_running_func(s):
    import time
    for i in range(10):
        time.sleep(1)
        print(f"Optimizing '{s}', step{i+1}")
    return f"Optimized {s}"

In [20]:
RunnableParallel(
    func1=long_running_func,
    func2=another_long_running_func,
).invoke("Hello world")

Processing 'Hello world', step1
Optimizing 'Hello world', step1
Processing 'Hello world', step2
Optimizing 'Hello world', step2
Processing 'Hello world', step3
Optimizing 'Hello world', step3
Processing 'Hello world', step4
Optimizing 'Hello world', step4
Optimizing 'Hello world', step5
Processing 'Hello world', step5
Optimizing 'Hello world', step6
Processing 'Hello world', step6
Optimizing 'Hello world', step7
Processing 'Hello world', step7
Processing 'Hello world', step8Optimizing 'Hello world', step8

Optimizing 'Hello world', step9
Processing 'Hello world', step9
Processing 'Hello world', step10
Optimizing 'Hello world', step10


{'func1': 'Processed Hello world', 'func2': 'Optimized Hello world'}

### @chain decorator

Another way to build chains is @chain decorator. It allows a more customable way to build chains.

In [21]:
# reimplement templated_llm using chain decorator

from langchain_core.runnables import RunnableConfig, chain

@chain
def templated_llm(user_input: str, config:RunnableConfig):
    value = template.invoke({"comment": user_input}, config=config)
    return llm.invoke(value, config=config)

In [22]:
templated_llm.invoke("I love this product, it is the best thing ever!")

AIMessage(content='Based on the user comment, I would rate the product a score of 5. The comment expresses strong positive feelings and satisfaction with the product.', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 30, 'prompt_tokens': 52, 'total_tokens': 82, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_d02d531b47', 'finish_reason': 'stop', 'logprobs': None}, id='run-598db142-a115-4924-bf6b-aa3927d5364c-0', usage_metadata={'input_tokens': 52, 'output_tokens': 30, 'total_tokens': 82, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 0}})

### Binding

Binding is used to add default invocation parameters to a runnable. For example, invoke of ChatOpenAI has a stop parameter to stop generating tokens. This can be set with bind.

In [23]:
template = ChatPromptTemplate.from_template(
    "Tell me 3 intersting facts about {subject}"
    )

In [24]:
templated_llm = template | llm

In [25]:
# output without stopping
out = templated_llm.invoke({"subject": "cats"})
print(out.content)

Sure! Here are three interesting facts about cats:

1. **Unique Communication**: Cats have a unique way of communicating with humans that differs from how they interact with other cats. While they use meowing primarily to communicate with humans, adult cats typically do not meow at each other. Instead, they use body language, purring, and other vocalizations to communicate with their feline peers.

2. **Whisker Sensitivity**: A cat's whiskers are highly sensitive tactile hairs called vibrissae. They are deeply embedded in the cat's skin and are packed with nerve endings, allowing cats to detect changes in their environment, navigate in the dark, and gauge whether they can fit through tight spaces. Whiskers can also help cats sense nearby objects and even changes in air currents.

3. **Sleep Patterns**: Cats are known for their love of sleep, and they can sleep anywhere from 12 to 16 hours a day, with some cats sleeping even more. This behavior is a result of their evolutionary history 

In [26]:
# binding stop parameter to llm
templated_llm = template | llm.bind(stop="\n")
out = templated_llm.invoke({"subject": "cats"})
print(out.content)

Sure! Here are three interesting facts about cats:


### Output Parsing

So far, we get AIMessage object as the output. We can parse the output to get the desired information. The simplest method is the get the string directly from the AIMessage object. 

In [27]:
from langchain_core.output_parsers import StrOutputParser

prompt_template = ChatPromptTemplate.from_template("tell me a joke about {topic}")
templated_llm = prompt_template | llm | StrOutputParser()

In [28]:
templated_llm.invoke("chicken")

'Why did the chicken join a band?\n\nBecause it had the drumsticks!'